In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
local_path_mnist_train = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_train.csv"
local_path_mnist_test = "/Users/gunnvantsaini/OneDrive/project_codes/content/dl_basics/vision/sony/data/mnist_test.csv"

In [3]:
mnist_train = pd.read_csv(local_path_mnist_train)
mnist_test = pd.read_csv(local_path_mnist_test)

In [4]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values

In [5]:
### Create a simple cnn
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(filters=6,kernel_size=(3,3),padding='same',input_shape=(28,28,1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Conv2D(filters=16,kernel_size=(3,3),padding='valid'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(120,activation='relu'))
model.add(tf.keras.layers.Dense(84,activation='relu'))
model.add(tf.keras.layers.Dense(10,activation='softmax'))

Metal device set to: Apple M1


2021-12-30 17:59:31.926050: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-12-30 17:59:31.926165: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 16)        880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               69240     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

In [7]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

In [8]:
X = X.reshape((X.shape[0],28,28,1))

In [9]:
X.shape

(42000, 28, 28, 1)

In [10]:
X = tf.constant(X,dtype='float32')
y = tf.constant(y,dtype='float32')
mnist_data = tf.data.Dataset.from_tensor_slices((X,y))

In [11]:
mnist_data = mnist_data.batch(64)

In [12]:
model.fit(mnist_data,epochs=10)

Epoch 1/10
  4/657 [..............................] - ETA: 11s - loss: 2.3132 - accuracy: 0.0352 

2021-12-30 17:59:32.376748: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-12-30 17:59:32.376817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


657/657 [==============================] - 7s 10ms/step - loss: 1.0633 - accuracy: 0.6855
Epoch 2/10
657/657 [==============================] - 6s 10ms/step - loss: 0.2656 - accuracy: 0.9185
Epoch 3/10
657/657 [==============================] - 6s 10ms/step - loss: 0.1923 - accuracy: 0.9409
Epoch 4/10
657/657 [==============================] - 7s 10ms/step - loss: 0.1537 - accuracy: 0.9531
Epoch 5/10
657/657 [==============================] - 7s 10ms/step - loss: 0.1296 - accuracy: 0.9611
Epoch 6/10
657/657 [==============================] - 7s 10ms/step - loss: 0.1126 - accuracy: 0.9661
Epoch 7/10
657/657 [==============================] - 7s 10ms/step - loss: 0.0999 - accuracy: 0.9699
Epoch 8/10
657/657 [==============================] - 6s 10ms/step - loss: 0.0901 - accuracy: 0.9726
Epoch 9/10
657/657 [==============================] - 6s 10ms/step - loss: 0.0821 - accuracy: 0.9749
Epoch 10/10
657/657 [==============================] - 6s 9ms/step - loss: 0.0755 - accuracy: 0.9770


**CNN**
- Padding,Filter Size and stride
- Lenet 5 architecture

The output from a convolution filter depends on the input size, kernel size, padding and stride. The relationship is:
$O=\frac{(I-K+2P)}{S}+1$ (https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/ )

With this in mind we will create a model for very common CNN architecture known as Lenet5 (https://engmrk.com/lenet-5-a-classic-cnn-architecture/)

In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
from torch import optim

In [14]:
def O(I,K,P,S):
    return ((I-K+2*P)/(S))+1

In [15]:
O(28,5,0,1)

24.0

In [16]:
O(24,2,0,2)

12.0

In [17]:
O(12,5,0,1)

8.0

In [18]:
O(8,2,0,2)

4.0

In [19]:
########### Define Model #########
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=6,padding=0,stride=1,kernel_size=(5,5))
        self.pool1=nn.MaxPool2d(kernel_size=(2,2),padding=0,stride=2)
        self.conv2=nn.Conv2d(in_channels=6,out_channels=16,padding=0,stride=1,kernel_size=(5,5))
        self.pool2=nn.MaxPool2d(kernel_size=(2,2),padding=0,stride=2)
        self.linear1=nn.Linear(4*4*16,120)
        self.linear2=nn.Linear(120,84)
        self.linear3=nn.Linear(84,10)
    def forward(self,X):
        x=self.conv1(X)
        x=nn.functional.tanh(x)
        x=self.pool1(x)
        x=self.conv2(x)
        x=nn.functional.tanh(x)
        x=self.pool2(x)
        x=x.view(-1,16*4*4)
        x=self.linear1(x)
        x=nn.functional.tanh(x)
        x=self.linear2(x)
        x=nn.functional.tanh(x)
        x=self.linear3(x)
        x=nn.functional.softmax(x,dim=1)
        return x        

In [20]:
class Mnist(Dataset):
    def __init__(self,X,y):
        self.X=X
        self.y=y
    def __len__(self):
        return len(self.y)
    def __getitem__(self,idx):
        X=self.X[idx,].reshape((28,28))
        y=self.y[idx]
        batch={'X':X,'y':y}
        return batch

In [21]:
X=mnist_train.drop('label',axis=1).values/255.0
y=mnist_train['label'].values

In [22]:
mnist=Mnist(X,y)

In [23]:
data=DataLoader(mnist,batch_size=128,shuffle=True)

In [24]:
criterion=nn.CrossEntropyLoss()

In [25]:
mod=Lenet()
opt=optim.Adam(mod.parameters(),lr=0.001)

In [27]:
num_epochs=4
Losses=[]
for i in range(num_epochs):
    for j,batch in enumerate(data):
        X=batch['X'].float()
        X=X.reshape(-1,1,28,28)
        y=batch['y'].long()
        probs=mod(X)
        loss=criterion(probs,y)
        Losses.append(loss.item())
        loss.backward()
        acc=(probs.argmax(axis=1)==y).float().mean().item()
        if j%100==0:
            print(f"Epoch {i+1}, iter {j+1}, acc {round(acc,2)}, loss {round(loss.item(),2)}")
        opt.step()
        opt.zero_grad()

/Users/gunnvantsaini/miniforge3/envs/dl/lib/python3.9/site-packages/torch/nn/functional.py:1795: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 1, iter 1, acc 0.14, loss 2.3
Epoch 1, iter 101, acc 0.89, loss 1.61
Epoch 1, iter 201, acc 0.95, loss 1.52
Epoch 1, iter 301, acc 0.95, loss 1.52
Epoch 2, iter 1, acc 0.95, loss 1.51
Epoch 2, iter 101, acc 0.94, loss 1.53
Epoch 2, iter 201, acc 0.97, loss 1.5
Epoch 2, iter 301, acc 0.98, loss 1.49
Epoch 3, iter 1, acc 0.96, loss 1.49
Epoch 3, iter 101, acc 0.98, loss 1.49
Epoch 3, iter 201, acc 0.98, loss 1.47
Epoch 3, iter 301, acc 0.95, loss 1.51
Epoch 4, iter 1, acc 0.98, loss 1.48
Epoch 4, iter 101, acc 0.98, loss 1.48
Epoch 4, iter 201, acc 0.99, loss 1.48
Epoch 4, iter 301, acc 0.99, loss 1.47


In [28]:
####### One can also use nn.sequential() to represent architectures better ######
class Lenet(nn.Module):
    def __init__(self):
        super().__init__()
        self.features=nn.Sequential(nn.Conv2d(in_channels=1,out_channels=6,padding=0,stride=1,kernel_size=(5,5)),
                                   nn.Tanh(),
                                   nn.MaxPool2d(kernel_size=(2,2),padding=0,stride=2),
                                   nn.Conv2d(in_channels=6,out_channels=16,padding=0,stride=1,kernel_size=(5,5)),
                                   nn.Tanh(),
                                   nn.MaxPool2d(kernel_size=(2,2),padding=0,stride=2))
        self.classifier=nn.Sequential(nn.Linear(4*4*16,120),
                                     nn.Tanh(),
                                     nn.Linear(120,84),
                                     nn.Tanh(),
                                     nn.Linear(84,10))
    def forward(self,X):
        X=self.features(X)
        X=X.view(-1,16*4*4)
        X=self.classifier(X)
        X=nn.functional.softmax(X,dim=1)
        return X

In [29]:
mod=Lenet()
opt=optim.Adam(mod.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [30]:
num_epochs=4
Losses=[]
for i in range(num_epochs):
    for j,batch in enumerate(data):
        X=batch['X'].float()
        X=X.reshape(-1,1,28,28)
        y=batch['y'].long()
        probs=mod(X)
        loss=criterion(probs,y)
        Losses.append(loss.item())
        loss.backward()
        acc=(probs.argmax(axis=1)==y).float().mean().item()
        if j%100==0:
            print(f"Epoch {i+1}, iter {j+1}, acc {round(acc,2)}, loss {round(loss.item(),2)}")
        opt.step()
        opt.zero_grad()

Epoch 1, iter 1, acc 0.1, loss 2.3
Epoch 1, iter 101, acc 0.91, loss 1.61
Epoch 1, iter 201, acc 0.92, loss 1.54
Epoch 1, iter 301, acc 0.95, loss 1.51
Epoch 2, iter 1, acc 0.95, loss 1.52
Epoch 2, iter 101, acc 0.98, loss 1.49
Epoch 2, iter 201, acc 0.97, loss 1.49
Epoch 2, iter 301, acc 0.93, loss 1.54
Epoch 3, iter 1, acc 0.97, loss 1.5
Epoch 3, iter 101, acc 0.99, loss 1.47
Epoch 3, iter 201, acc 0.97, loss 1.5
Epoch 3, iter 301, acc 0.95, loss 1.52
Epoch 4, iter 1, acc 0.98, loss 1.48
Epoch 4, iter 101, acc 0.97, loss 1.5
Epoch 4, iter 201, acc 0.95, loss 1.5
Epoch 4, iter 301, acc 1.0, loss 1.47
